## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import measure
from skimage.measure import regionprops, regionprops_table
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import load_img
from importlib import reload
import segmenteverygrain as seg
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from tqdm import trange
%matplotlib qt

## Load models

In [2]:
model = seg.Unet()
model.compile(optimizer=Adam(), loss=seg.weighted_crossentropy, metrics=["accuracy"])
model.load_weights('./checkpoints/seg_model') # replace this if you have a finetuned Unet model and want to use it

# the SAM model checkpoints can be downloaded from: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam = sam_model_registry["default"](checkpoint="/Users/zoltan/Dropbox/Segmentation/sam_vit_h_4b8939.pth")

## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images, so do not start with large images (downsample them if necessary). Images with ~2000 pixels along their largest dimension are a good start.

In [45]:
reload(seg)

<module 'segmenteverygrain' from '/Users/zoltan/Dropbox/Segmentation/segmenteverygrain/segmenteverygrain/segmenteverygrain.py'>

In [46]:
# replace this with the path to your image:
fname = '/Users/zoltan/Dropbox/Segmentation/torrey_pines_beach.jpeg'
big_im = np.array(load_img(fname))
big_im_pred = seg.predict_big_image(big_im, model, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
labels, coords = seg.label_grains(big_im, big_im_pred, dbs_max_dist=20.0) # Unet prediction

# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, big_im, big_im_pred, 
                                                coords, labels, min_area=400.0, plot_image=True, remove_edge_grains=False, remove_large_objects=False)

segmenting image tiles...


100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.13it/s]


creating masks using SAM...


100%|██████████████████████████████████████████████████████████████████████████████| 995/995 [00:40<00:00, 24.83it/s]


finding overlapping polygons...


992it [00:02, 459.31it/s]


finding best polygons...


100%|█████████████████████████████████████████████████████████████████████████████| 380/380 [00:01<00:00, 271.73it/s]


creating labeled image...


Use this figure to check the distribution of SAM prompts (= black dots):

In [13]:
plt.figure(figsize=(15,10))
plt.imshow(big_im_pred)
plt.scatter(np.array(coords)[:,0], np.array(coords)[:,1], c='k')
plt.xticks([])
plt.yticks([]);

## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [64]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
                              lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [37]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'all_grains' list after deleting and merging grains:

In [44]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

Plot the updated set of grains:

In [43]:
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
# seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [208]:
fname = "/Users/zoltan/Dropbox/Permian_Basin/Thin_sections_2023/J4-7228.0_10x_upper_part_reduced_size.jpg"
big_im = np.array(load_img(fname))
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])

([], [])

In [209]:
predictor = SamPredictor(sam)
predictor.set_image(big_im) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.onclick(event, ax, coords, big_im, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event', lambda event: seg.onpress(event, ax, fig))

In [205]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

In [ ]:
reload(seg)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [57]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|█████████████████████████████████████| 10081/10081 [42:25<00:00,  3.96it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [151]:
cid5 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.click_for_scale(event, ax))

number of pixels: 773.27


Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [152]:
n_of_units = 500
units_per_pixel = n_of_units/773.27 # length of scale bar in pixels

In [156]:
props = regionprops_table(rasterized.astype('int'), intensity_image = big_im, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values*units_per_pixel
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values*units_per_pixel
grain_data['perimeter'] = grain_data['perimeter'].values*units_per_pixel
grain_data['area'] = grain_data['area'].values*units_per_pixel**2

In [697]:
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,210.659649,1106.354806,38.431493,20.201423,13.974698,1.184300,58.038774,221.0,223.0,224.0,158.148262,157.610429,150.855828,61.0,60.0,72.0
1,2,107.268410,554.865462,60.995984,18.078866,7.772253,-1.303846,43.055309,252.0,255.0,255.0,179.953815,190.333333,189.985944,59.0,52.0,72.0
2,3,31.878965,774.020270,43.189189,7.660721,5.315649,-1.363193,20.327564,243.0,255.0,239.0,183.344595,194.283784,186.655405,38.0,94.0,87.0
3,4,178.996082,128.078219,407.138387,24.902167,10.000839,1.187134,59.895215,235.0,242.0,231.0,178.498195,178.044525,168.651023,67.0,73.0,69.0
4,5,122.346299,141.345070,699.503521,15.353942,10.284223,-1.113555,42.511570,246.0,255.0,255.0,172.940141,177.843310,180.357394,64.0,62.0,93.0


In [158]:
grain_data.to_csv(fname[:-4]+'.csv')

In [161]:
plt.figure()
plt.hist(grain_data['major_axis_length'], np.arange(0, 100, 1), alpha=0.5)
plt.hist(grain_data['minor_axis_length'], np.arange(0, 100, 1), alpha=0.5)
plt.xlim(0,100)
plt.xlabel('axis length (microns)')
plt.ylabel('count');

## Save mask and grain labels to PNG files

In [58]:
dirname = '/Users/zoltan/Dropbox/Segmentation/images/'
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_mask.png', mask_all)
# Save the image as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_image.png', cv2.cvtColor(big_im, cv2.COLOR_BGR2RGB))

True

## Run segmentation on large image (new!)
In this case 'fname' points to an image that is larger than a few megapixels and has thousands of grains.
The 'predict_large_image' function breaks the input image into smaller patches and it runs the segmentation process on each patch.

In [33]:
reload(seg)
from PIL import Image
Image.MAX_IMAGE_PIXELS = None # needed if working with very large images
fname = "/Users/zoltan/Dropbox/Segmentation/images/mair_et_al_L2_DJI_0382_image.png"
all_grains = seg.predict_large_image(fname, model, sam, min_area=400.0, patch_size=2000, overlap=200)

segmenting image tiles...


100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.30it/s]


creating masks using SAM...


100%|████████████████████████████████████████████████████████████████████████████| 3197/3197 [03:25<00:00, 15.54it/s]


finding overlapping polygons...


...

1712it [00:06, 257.94it/s]


finding best polygons...


100%|██████████████████████████████████████████████████████████████████████████████| 508/508 [00:06<00:00, 74.57it/s]


creating labeled image...


4731it [00:02, 2242.65it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 319/319 [00:00<00:00, 881.09it/s]


In [34]:
# plot results
big_im = np.array(load_img(fname))
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
plt.axis('equal')
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

In [40]:
# this is a faster way of deleting false positives (because it avoids highlighting and deleting the grains)
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax, select_only=True))

In [42]:
# delete polygons from 'all_grains'
grain_inds = np.unique(grain_inds)
grain_inds = sorted(grain_inds, reverse=True)
for ind in grain_inds:
    all_grains.remove(all_grains[ind])

In [ ]:
fig.canvas.mpl_disconnect(cid1)

In [ ]:
# plot image with updated grains
big_im = np.array(load_img(fname))
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
plt.axis('equal')
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

In [ ]:
# collect data from plot, including mask
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

In [140]:
from skimage.measure import regionprops, regionprops_table
props = regionprops_table(labels, intensity_image = big_im, properties=('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
                                                                                'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))

In [142]:
df = pd.DataFrame(props)

In [143]:
df.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,830.0,43960.320482,1677.862651,38.752309,27.316355,0.273334,107.497475,200.0,205.0,214.0,77.914458,67.026506,61.689157,10.0,25.0,20.0
1,2,1136.0,43590.179577,686.280810,43.211267,34.025389,0.857311,126.225397,207.0,218.0,211.0,145.168134,143.100352,143.987676,34.0,37.0,65.0
2,3,1571.0,43189.798218,963.990452,56.501983,35.768381,0.805265,149.923882,217.0,222.0,216.0,149.928708,145.283896,141.232336,26.0,31.0,21.0
3,4,3388.0,43150.272137,588.850945,92.152532,47.986861,1.432245,237.823376,226.0,224.0,226.0,144.553424,139.338843,133.357143,32.0,41.0,39.0
4,5,897.0,42856.816054,620.888517,42.509864,27.112746,-1.446275,114.568542,223.0,223.0,216.0,177.035674,176.288740,176.924192,68.0,59.0,67.0


In [195]:
# plot grains in black and white
plt.figure()
# plt.imshow(big_im[0:2001, 0:2001, :])
for grain in tqdm(all_grains):
    plt.plot(grain.exterior.xy[0], grain.exterior.xy[1], 'k', linewidth=0.2)
plt.axis('equal')
plt.gca().invert_yaxis();

100%|███████████████████████████████████| 78923/78923 [00:21<00:00, 3616.88it/s]


After plotting the results, you will want to use the functions for deleting, merging, and adding grains (see above), before saving the results (same workflow as for a small image).